In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [2]:
loader = CSVLoader('cleaned_judgements.csv')

In [3]:
my_cs=loader.load()

In [4]:
print(my_cs[0])

page_content='Year: 1950
Petitioner: PANNALAL JANKIDAS
Respondent: MOHANLAL AND ANOTHER.' metadata={'source': 'cleaned_judgements.csv', 'row': 0}


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,
    chunk_overlap=500,
)

In [7]:
my_chunk=text_splitter.split_documents(my_cs)

In [8]:
my_chunk[0].page_content

'Year: 1950\nPetitioner: PANNALAL JANKIDAS\nRespondent: MOHANLAL AND ANOTHER.'

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [10]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [11]:
vector_store.add_documents(documents=my_chunk)

['63eff315-8b5d-4bce-aee8-1d7ec7d74489',
 '02ac8751-bb34-4bba-902f-90eac59589f7',
 '500c7b20-771d-462a-aa46-77993b8d76bf',
 'a2a4ccbc-5e2d-4adf-bce9-a9ec13ef05c2',
 'a09506c7-ef1d-48ab-a0ad-6c35fd972530',
 'a43cac41-a6aa-4e3b-8256-6b9fc8675095',
 '802abfb2-cfdd-4bb6-bbcd-79ab36fa05e4',
 '71e7b03d-3b35-4d87-a08a-c19d9c1aaaeb',
 '93903511-e73b-46c7-9080-aa7cd81fd4e2',
 'adc6a279-2489-4006-b584-0f0e050e39de',
 '14800a0e-f484-409f-914b-7d8d82f5cafb',
 '3af8e95e-a59c-4967-b0c1-bf2a2b5f709a',
 '03a8aabf-ccc3-4ec4-bf00-84aa9172fd14',
 'd3c0e5de-0339-47bd-bf39-65b2e499a1e3',
 '42b11016-6125-40d1-835e-835d09bc7913',
 'bc1278af-8336-4826-bc79-b5d1e7fac2c0',
 'c1bbe1b4-8ce2-4fc0-9581-8f1d0e7d4794',
 '82ed5517-eb05-4a6e-ac2c-754d17c0d758',
 '296cf6ed-4eb2-4657-82ff-50c2c8565379',
 '517da3a8-c495-49f6-bea7-3ce1fdaa9711',
 '6a2069f4-9d54-443e-b4b1-04b827ca452a',
 '38923d1b-72df-4b01-a663-88f5ebb0ef07',
 '52a8c5e7-1887-47d7-9ba6-406a81e8d310',
 '9678522f-e17e-4b45-8f38-022f820de497',
 'f30bd1b8-cacb-

In [12]:
context=vector_store.similarity_search("Year of PANNALAL JANKIDAS and MOHANLAL AND ANOTHER",k=2)

In [13]:
actual_content=context[0].page_content
print(actual_content)

Year: 1950
Petitioner: PANNALAL JANKIDAS
Respondent: MOHANLAL AND ANOTHER.


In [14]:
our_ques="Year of PANNALAL JANKIDAS and MOHANLAL AND ANOTHER"
our_prompt = f"""
You are an expert legal assistant.
Return the answer as a bullet list of case names with their year.
If no relevant information is available, 
Here is the context of some Supreme Court cases:{actual_content}
Our Question:{our_ques}

"""

In [15]:
print(our_prompt)


You are an expert legal assistant.
Return the answer as a bullet list of case names with their year.
If no relevant information is available, 
Here is the context of some Supreme Court cases:Year: 1950
Petitioner: PANNALAL JANKIDAS
Respondent: MOHANLAL AND ANOTHER.
Our Question:Year of PANNALAL JANKIDAS and MOHANLAL AND ANOTHER




In [16]:
from openai import OpenAI

user_input = our_prompt

client = OpenAI(
    base_url="http://127.0.0.1:1234/v1",
    api_key="lm-studio"
)

response = client.chat.completions.create(
    model="mistral-7b-instruct-v0.1",
    messages=[
        {"role": "user", "content": user_input}
    ]
)

print("Assistant:", response.choices[0].message.content)


Assistant: 
- PANNALAL JANKIDAS v. MOHANLAL AND ANOTHER (1950)
